You now know two kinds of regression and two kinds of classifier. So let's use that to compare models!

Comparing models is something data scientists do all the time. There's very rarely just one model that would be possible to run for a given situation, so learning to choose the best one is very important.

Here let's work on regression. Find a data set and build a KNN Regression and an OLS regression. Compare the two. How similar are they? Do they miss in different ways?

Create a Jupyter notebook with your models. At the end in a markdown cell write a few paragraphs to describe the models' behaviors and why you favor one model or the other. Try to determine whether there is a situation where you would change your mind, or whether one is unambiguously better than the other. Lastly, try to note what it is about the data that causes the better model to outperform the weaker model. Submit a link to your notebook below.

In [1]:
# Import data science environmemnt.
import math
import warnings

from IPython.display import display
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import neighbors
from sklearn import linear_model 
from sklearn.model_selection import cross_val_score
import statsmodels.formula.api as smf

# Display preferences
% matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error
warnings.filterwarnings(
    action='ignore',
    module='scipy',
    message='^internal gelsd'
)
warnings.filterwarnings('ignore')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv', skiprows=3, header=1)
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan


In [3]:
# Change long column names.
df.rename(columns={"Murder and\nnonnegligent\nmanslaughter":"Murder"})

,City,Population,Violent crime,Murder,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan
5,Alfred Village,"4,089",5,0.000,nan,0,3,2,46,10,36,0,nan
6,Allegany Village,"1,781",3,0.000,nan,0,0,3,10,0,10,0,0.000
7,Amherst Town,"118,296",107,1.000,nan,7,31,68,"2,118",204,"1,882",32,3.000
8,Amityville Village,"9,519",9,0.000,nan,2,4,3,210,16,188,6,1.000
9,Amsterdam,"18,182",30,0.000,nan,0,12,18,405,99,291,15,0.000


In [4]:
# Eliminate commas from number > 999.
def convert_number(number):
    try:
        converted = float(number.replace(',', ''))
    except:
        converted = number
        
    return converted

In [5]:
# Change NaN values to 0. Convert object types to floats.
df.dropna()
df['Population'] = df['Population'].apply(lambda x: convert_number(x))
df['Population^2'] = df['Population']**2
df['Murder'] = df['Murder and\nnonnegligent\nmanslaughter'].apply(lambda x: convert_number(x))
df['Robbery'] = df['Robbery'].apply(lambda x: convert_number(x))
df['Property_Crime'] = df['Property\ncrime'].apply(lambda x: convert_number(x))

In [6]:
# Eliminate final three rows of text after data.
df = df[:348]

In [7]:
# Create new data frame with only relevant columns.
df_fbi = df[['City', 'Population', 'Population^2', 'Murder', 'Robbery', 'Property_Crime']]

In [8]:
# Preview new data frame.
df_fbi.head()

,City,Population,Population^2,Murder,Robbery,Property_Crime
0,Adams Village,1861.000,3463321.000,0.000,0.000,12.000
1,Addison Town and Village,2577.000,6640929.000,0.000,0.000,24.000
2,Akron Village,2846.000,8099716.000,0.000,0.000,16.000
3,Albany,97956.000,9595377936.000,8.000,227.000,4090.000
4,Albion Village,6388.000,40806544.000,0.000,4.000,223.000


In [18]:
# Build our KNN model.
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
X = df_fbi.Population
Y = df_fbi.Property_Crime.values.reshape(-1, 1)
knn.fit(X, Y)

# Set up our prediction line.
T = np.arange(0, 20000, 100)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
Y = knn.predict(T)

plt.scatter(X, Y, c='k', label='data')
plt.plot(T, Y_, c='g', label='prediction')
plt.legend()
plt.title('K=5, Unweighted')
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[1.861000e+03 2.577000e+03 2.846000e+03 9.795600e+04 6.388000e+03
 4.089000e+03 1.781000e+03 1.182960e+05 9.519000e+03 1.818200e+04
 2.053000e+03 4.523000e+03 6.580000e+02 2.522000e+03 2.727000e+04
 3.333000e+03 7.473000e+03 5.418000e+03 1.537400e+04 5.733000e+03
 1.762700e+04 3.424300e+04 4.630400e+04 1.410000e+03 1.200100e+04
 1.022000e+03 2.325000e+03 2.054000e+03 2.065000e+03 2.358000e+03
 7.976000e+03 3.668900e+04 8.284000e+03 6.390000e+03 1.174000e+03
 2.587890e+05 6.577000e+03 1.854000e+03 2.219000e+03 2.416500e+04
 1.047000e+04 4.704000e+03 2.246000e+03 6.794000e+03 7.620000e+02
 3.446200e+04 3.481000e+03 3.862000e+03 9.800000e+02 3.749000e+03
 2.732000e+03 1.835000e+03 1.733000e+03 7.836100e+04 7.990000e+03
 5.016000e+03 2.957100e+04 8.070500e+04 2.049000e+03 4.503000e+03
 7.458000e+03 1.617900e+04 7.821500e+04 1.827000e+03 1.108700e+04
 2.978000e+03 9.543000e+03 1.933100e+04 9.264000e+03 8.180000e+03
 3.209000e+03 4.6